<a href="https://colab.research.google.com/github/TalhaOmi/unet_finger_vein_tensor_federated/blob/main/unet_with_tensor(federated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Model

In [10]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet_main(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

if __name__ == "__main__":
  input_shape = (512, 512, 3)
  model_main = build_unet_main(input_shape)
  model_main.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 512, 512, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 512, 512, 64  256        ['conv2d_38[0][0]']              
 ormalization)                  )                                                              

# Metrics

In [11]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [12]:
#with open("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/test", "rb") as fp:   # Unpickling
#   b = pickle.load(fp)

# Train

In [13]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "set1","image", "*")))
    y = sorted(glob(os.path.join(path, "set1","mask", "*")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4
    num_epochs = 20
    model_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "model.h5")
    csv_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "data.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model_main = build_unet_main((H, W, 3))
    #print(model_main.get_weights())
    #model_main.set_weights(b)
    model_main.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    #model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model_main.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 19 - 19
Valid: 5 - 5
Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.9625 - dice_coef: 0.0375 - iou: 0.0192 - recall_1: 0.4115 - precision_1: 0.3359
Epoch 1: val_loss improved from inf to 0.97905, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model.h5
19/19 [==============================] - 10s 400ms/step - loss: 0.9625 - dice_coef: 0.0375 - iou: 0.0192 - recall_1: 0.4115 - precision_1: 0.3359 - val_loss: 0.9790 - val_dice_coef: 0.0210 - val_iou: 0.0106 - val_recall_1: 1.0000 - val_precision_1: 0.4030 - lr: 1.0000e-04
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.9515 - dice_coef: 0.0485 - iou: 0.0249 - recall_1: 0.3159 - precision_1: 0.3804
Epoch 2: val_loss improved from 0.97905 to 0.97903, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model.h5
19/19 [==============================] - 7s 368ms/step - loss: 0.9515 - dice_coef: 0.0485 - iou: 0.0249 - recall_1: 0.3159 - 

In [14]:
main_model_weight=model_main.get_weights()

# Test

In [15]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from sklearn.metrics import classification_report


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    y = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_y = y
    y = y/255.0
    y = y.astype(np.int32)
    return ori_y, y



def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Save the results in this folder """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/results1")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model.h5")

    """ Load the dataset """
    test_x = sorted(glob("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/test/image/*"))
    test_y = sorted(glob("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/test/mask/*"))

    """ Make the prediction and calculate the metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracting name """
        name = x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)

        """ Saving the images """
        save_image_path = f"/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/results1/{name}.png"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred,labels=[0, 1], average="macro")
        jac_value = jaccard_score(y, y_pred,labels=[0, 1], average="macro")
        recall_value = recall_score(y, y_pred,labels=[0, 1], average="macro")
        precision_value = precision_score(y, y_pred,labels=[0, 1], average="macro")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print()
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")



 10%|█         | 1/10 [00:00<00:07,  1.27it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 20%|██        | 2/10 [00:01<00:04,  1.76it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 30%|███       | 3/10 [00:01<00:03,  2.03it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r


Accuracy: 0.99896
F1: 0.49974
Jaccard: 0.49948
Recall: 0.49948
Precision: 0.50000


# Model 1(test)

In [16]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet_1(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model
    

In [17]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "set2","image", "*")))
    y = sorted(glob(os.path.join(path, "set2","mask", "*")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4
    num_epochs = 20
    model_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "model1.h5")
    csv_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "data1.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model1 = build_unet_1((H, W, 3))
    model1.set_weights(main_model_weight)
    model1.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model1.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 19 - 19
Valid: 5 - 5
Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.9157 - dice_coef: 0.0843 - iou: 0.0441 - recall_2: 0.2049 - precision_2: 0.5225
Epoch 1: val_loss improved from inf to 0.97587, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model1.h5
19/19 [==============================] - 29s 1s/step - loss: 0.9157 - dice_coef: 0.0843 - iou: 0.0441 - recall_2: 0.2049 - precision_2: 0.5225 - val_loss: 0.9759 - val_dice_coef: 0.0241 - val_iou: 0.0122 - val_recall_2: 0.1232 - val_precision_2: 0.0946 - lr: 1.0000e-04
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.9023 - dice_coef: 0.0977 - iou: 0.0514 - recall_2: 0.2054 - precision_2: 0.5752
Epoch 2: val_loss improved from 0.97587 to 0.96789, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model1.h5
19/19 [==============================] - 7s 380ms/step - loss: 0.9023 - dice_coef: 0.0977 - iou: 0.0514 - recall_2: 0.2054 - p

In [18]:
test1_model_weight = model1.get_weights()

# Model2 (test)

In [19]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet_2(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model
    

In [20]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "set3","image", "*")))
    y = sorted(glob(os.path.join(path, "set3","mask", "*")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4
    num_epochs = 20
    model_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "model2.h5")
    csv_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "data2.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model2 = build_unet_2((H, W, 3))
    model2.set_weights(main_model_weight)
    model2.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model1.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 19 - 19
Valid: 5 - 5
Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.8731 - dice_coef: 0.1269 - iou: 0.0680 - recall_2: 0.2288 - precision_2: 0.4885
Epoch 1: val_loss improved from inf to 0.90304, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model2.h5
19/19 [==============================] - 22s 1s/step - loss: 0.8731 - dice_coef: 0.1269 - iou: 0.0680 - recall_2: 0.2288 - precision_2: 0.4885 - val_loss: 0.9030 - val_dice_coef: 0.0970 - val_iou: 0.0510 - val_recall_2: 0.3102 - val_precision_2: 0.4442 - lr: 1.0000e-04
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.8404 - dice_coef: 0.1596 - iou: 0.0869 - recall_2: 0.2609 - precision_2: 0.5912
Epoch 2: val_loss did not improve from 0.90304
19/19 [==============================] - 6s 306ms/step - loss: 0.8404 - dice_coef: 0.1596 - iou: 0.0869 - recall_2: 0.2609 - precision_2: 0.5912 - val_loss: 0.9646 - val_dice_coef: 0.0354 - val_iou: 0.0181 - val_recall_

In [21]:
test2_model_weight = model2.get_weights()

In [22]:
addition = [x + y for (x, y) in zip(test1_model_weight, test2_model_weight)] 

In [23]:
average_weight = [x / 2 for x in addition]

In [24]:
test1_model_weight[1]

array([-2.5553283e-05,  1.0903518e-05, -4.5498609e-04, -1.3972581e-03,
        1.2187082e-03, -1.0818432e-03, -3.7178395e-03, -5.3266118e-05,
        1.8180141e-04, -1.6808416e-03, -3.1459488e-03,  6.4421189e-04,
       -2.8726330e-03,  6.0499879e-04, -1.5771989e-03, -9.2214199e-05,
        5.6727428e-04, -2.1721255e-04,  3.6450175e-03,  7.6650955e-05,
       -2.0863409e-03, -1.9540582e-03, -3.1556194e-03, -2.1723111e-04,
       -1.8162451e-05,  2.5621994e-04, -2.3599612e-03,  1.2184231e-04,
        2.0281810e-03, -1.3034323e-03,  6.9877418e-04,  6.6290126e-04,
       -4.8778056e-06, -1.8893725e-05,  1.0964203e-03,  1.3400710e-04,
        1.5873984e-03,  8.7676418e-04,  3.9802809e-04,  1.2219038e-03,
        3.4696259e-05, -2.5322225e-03,  1.0542824e-03,  1.3516650e-03,
       -7.9937257e-05,  2.5385962e-04,  1.1535315e-05,  1.0736098e-04,
        1.2111211e-03, -3.2475991e-03, -4.7778900e-04,  9.3452487e-05,
       -1.7203639e-04, -1.4624388e-03,  1.1295434e-03, -9.5851300e-04,
      

In [25]:
test2_model_weight[1]

array([-1.77368158e-04, -5.95017846e-05,  1.56905531e-04,  6.63477927e-04,
       -8.26787378e-04, -3.80951707e-04,  6.40103943e-04,  1.44280406e-04,
        1.96476627e-04, -1.70962105e-03, -8.85390502e-04,  8.51202523e-04,
        4.22853482e-04,  1.61454358e-04,  1.96811612e-04,  1.02018654e-04,
        9.85602732e-04, -4.53178305e-04,  1.12265942e-03,  8.17199907e-05,
        1.20600067e-04, -9.76447773e-04, -1.17283536e-03, -7.12048277e-05,
        5.30514553e-05, -4.83056783e-06, -1.39297044e-03, -1.70432177e-04,
       -9.30314476e-04, -1.86455625e-04,  6.83442995e-05, -1.15152914e-03,
       -4.21561330e-04, -9.12194955e-05,  2.70050252e-04, -1.54758207e-04,
       -1.23072741e-05, -2.88991287e-04,  1.21419587e-04,  1.21155242e-03,
       -1.10057645e-05, -8.84760462e-04,  4.55492904e-04,  1.14833652e-04,
        1.09752938e-04, -1.45919490e-04,  2.62234651e-04,  7.33638735e-05,
       -4.03415062e-04,  5.85109694e-04, -1.38250834e-04, -1.16294203e-03,
        6.49475260e-05, -

In [26]:
average_weight[1]

array([-1.01460719e-04, -2.42991337e-05, -1.49040279e-04, -3.66890104e-04,
        1.95960427e-04, -7.31397420e-04, -1.53886783e-03,  4.55071422e-05,
        1.89139013e-04, -1.69523130e-03, -2.01566960e-03,  7.47707207e-04,
       -1.22488977e-03,  3.83226579e-04, -6.90193614e-04,  4.90222737e-06,
        7.76438508e-04, -3.35195422e-04,  2.38383841e-03,  7.91854691e-05,
       -9.82870464e-04, -1.46525295e-03, -2.16422742e-03, -1.44217964e-04,
        1.74445013e-05,  1.25694685e-04, -1.87646574e-03, -2.42949318e-05,
        5.48933283e-04, -7.44943973e-04,  3.83559236e-04, -2.44313938e-04,
       -2.13219566e-04, -5.50566110e-05,  6.83235296e-04, -1.03755519e-05,
        7.87545519e-04,  2.93886464e-04,  2.59723834e-04,  1.21672812e-03,
        1.18452472e-05, -1.70849147e-03,  7.54887646e-04,  7.33249355e-04,
        1.49078405e-05,  5.39700632e-05,  1.36884977e-04,  9.03624314e-05,
        4.03853017e-04, -1.33124471e-03, -3.08019924e-04, -5.34744759e-04,
       -5.35444342e-05, -

# Repeat with average weight in main model

In [27]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet_main(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

if __name__ == "__main__":
  input_shape = (512, 512, 3)
  model_main = build_unet_main(input_shape)
  

# Train

In [28]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "set1","image", "*")))
    y = sorted(glob(os.path.join(path, "set1","mask", "*")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4
    num_epochs = 20
    model_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "model3.h5")
    csv_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "data3.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model_main = build_unet_main((H, W, 3))
    model_main.set_weights(average_weight)
    model_main.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model_main.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 19 - 19
Valid: 5 - 5
Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.8625 - dice_coef: 0.1375 - iou: 0.0739 - recall_4: 0.0804 - precision_4: 0.6404
Epoch 1: val_loss improved from inf to 0.96437, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model3.h5
19/19 [==============================] - 19s 887ms/step - loss: 0.8625 - dice_coef: 0.1375 - iou: 0.0739 - recall_4: 0.0804 - precision_4: 0.6404 - val_loss: 0.9644 - val_dice_coef: 0.0356 - val_iou: 0.0182 - val_recall_4: 0.0118 - val_precision_4: 0.9034 - lr: 1.0000e-04
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.8434 - dice_coef: 0.1566 - iou: 0.0851 - recall_4: 0.0804 - precision_4: 0.6963
Epoch 2: val_loss did not improve from 0.96437
19/19 [==============================] - 6s 308ms/step - loss: 0.8434 - dice_coef: 0.1566 - iou: 0.0851 - recall_4: 0.0804 - precision_4: 0.6963 - val_loss: 0.9694 - val_dice_coef: 0.0306 - val_iou: 0.0155 - val_reca

In [29]:
#main_model_weight=model_main.get_weights()

# Test

In [30]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from sklearn.metrics import classification_report


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    y = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_y = y
    y = y/255.0
    y = y.astype(np.int32)
    return ori_y, y



def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Save the results in this folder """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/results2")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model3.h5")

    """ Load the dataset """
    test_x = sorted(glob("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/test/image/*"))
    test_y = sorted(glob("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/test/mask/*"))

    """ Make the prediction and calculate the metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracting name """
        name = x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)

        """ Saving the images """
        save_image_path = f"/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/results2/{name}.png"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred,labels=[0, 1], average="macro")
        jac_value = jaccard_score(y, y_pred,labels=[0, 1], average="macro")
        recall_value = recall_score(y, y_pred,labels=[0, 1], average="macro")
        precision_value = precision_score(y, y_pred,labels=[0, 1], average="macro")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print()
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")



 10%|█         | 1/10 [00:01<00:13,  1.51s/it]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 20%|██        | 2/10 [00:02<00:08,  1.05s/it]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 30%|███       | 3/10 [00:03<00:06,  1.08it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r


Accuracy: 0.98651
F1: 0.49672
Jaccard: 0.49331
Recall: 0.53611
Precision: 0.50006


In [31]:
import pickle

with open("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/test", "wb") as fp:   #Pickling
   pickle.dump(average_weight, fp)
 

In [32]:
with open("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/test", "rb") as fp:   # Unpickling
   loaded_weight = pickle.load(fp)
   
loaded_weight[1]

array([-1.01460719e-04, -2.42991337e-05, -1.49040279e-04, -3.66890104e-04,
        1.95960427e-04, -7.31397420e-04, -1.53886783e-03,  4.55071422e-05,
        1.89139013e-04, -1.69523130e-03, -2.01566960e-03,  7.47707207e-04,
       -1.22488977e-03,  3.83226579e-04, -6.90193614e-04,  4.90222737e-06,
        7.76438508e-04, -3.35195422e-04,  2.38383841e-03,  7.91854691e-05,
       -9.82870464e-04, -1.46525295e-03, -2.16422742e-03, -1.44217964e-04,
        1.74445013e-05,  1.25694685e-04, -1.87646574e-03, -2.42949318e-05,
        5.48933283e-04, -7.44943973e-04,  3.83559236e-04, -2.44313938e-04,
       -2.13219566e-04, -5.50566110e-05,  6.83235296e-04, -1.03755519e-05,
        7.87545519e-04,  2.93886464e-04,  2.59723834e-04,  1.21672812e-03,
        1.18452472e-05, -1.70849147e-03,  7.54887646e-04,  7.33249355e-04,
        1.49078405e-05,  5.39700632e-05,  1.36884977e-04,  9.03624314e-05,
        4.03853017e-04, -1.33124471e-03, -3.08019924e-04, -5.34744759e-04,
       -5.35444342e-05, -